<a href="https://colab.research.google.com/github/sipocz/_earthQuake/blob/main/_earthquake_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#_Dengue

_PCVERSION_=False


if _PCVERSION_:
    basedir="C:/Users/sipocz/OneDrive/Dokumentumok/GitHub"
else:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    basedir="/content/drive/My Drive/001_AI"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#--------------scikit import 
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#--------------
def outlierStatistic(X_train_predict):
    print(X_train_predict)
    maxX=len(X_train_predict)
    outlier=0
    for i in X_train_predict:
        if i==-1:
            outlier+=1
    print(f"A összes ({maxX} darabból {outlier} darab outlier van. Az {outlier/maxX*100:5.1f} %.)")


def checkvalues(df,columnname,key):
    print(f"{columnname} ellenőrzése !")
    numok=0
    numerr=0
    for i in df.index:
        if df.at[i,columnname] in key:
            #print(df.at[i,columnname])
            numok+=1
            pass
        else:
            numerr+=1
            print(df.at[i,columnname],end=", ")
    sumall=numok+numerr
    print(f"\n{sumall} mintából {numerr} db nem volt megfelelő")


def createcolumn(df,columnname,keys):
    print(f"{columnname} cseréje megy")
    for key in keys:
        df[keys[key]]=0.0
    for key in keys:
        for i in df.index:
            if df.at[i,columnname]==key:
                df.at[i,keys[key]]=1.0


def create_dict(idx,list):
    o={}
    for i in list:
        o[i]=idx+"_"+i
    return o

def create_base_data(df):
    t=['n', 't', 'o']
    columnname="land_surface_condition"
    key=create_dict(columnname,t)


    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t= ['h', 'w', 'i', 'r', 'u']
    columnname="foundation_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=  ['q', 'n', 'x']
    columnname="roof_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=  ['z', 'v', 'f', 'm', 'x']
    columnname="ground_floor_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['q', 's', 'j', 'x']
    columnname="other_floor_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['j', 's', 't', 'o']
    columnname="position"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['c', 's', 'f', 'd', 'm', 'a', 'q', 'u', 'n', 'o']
    columnname="plan_configuration"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=['a', 'w', 'r', 'v']
    columnname="legal_ownership_status"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    df["geopos"]=df.geo_level_1_id+df.geo_level_2_id/10000+df.geo_level_3_id/1000000000
    return(df)



def kill_columns(df):
    notkey=["building_id","legal_ownership_status","geo_level_1_id",	"geo_level_2_id",	"geo_level_3_id", "land_surface_condition",	"foundation_type",	"roof_type",	"ground_floor_type",	"other_floor_type",	"position",	"plan_configuration"]
    for i in df.columns:
        if i in notkey:
            df.drop(columns=[i], inplace=True)
    return df

f=open(basedir+"/_EarthQuake/similarity.csv","a")
X_train_ok=pd.read_csv(basedir+"/_EarthQuake/X_tran_ok.csv")
X_pred_ok=pd.read_csv(basedir+"/_EarthQuake/X_pred_ok.csv")
y_train_ok=pd.read_csv(basedir+"/_EarthQuake/y_train_ok.csv")

X_train_ok.head()
X_pred_ok.head()
print("hello")





In [3]:
X_pred_ok.head()

,Unnamed: 0,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_t,land_surface_condition_o,foundation_type_h,foundation_type_w,foundation_type_i,foundation_type_r,foundation_type_u,roof_type_q,roof_type_n,roof_type_x,ground_floor_type_z,ground_floor_type_v,ground_floor_type_f,ground_floor_type_m,ground_floor_type_x,other_floor_type_q,other_floor_type_s,other_floor_type_j,other_floor_type_x,position_j,position_s,position_t,position_o,plan_configuration_c,plan_configuration_s,plan_configuration_f,plan_configuration_d,plan_configuration_m,plan_configuration_a,plan_configuration_q,plan_configuration_u,plan_configuration_n,plan_configuration_o,legal_ownership_status_a,legal_ownership_status_w,legal_ownership_status_r,legal_ownership_status_v,geopos
0,0,3,20,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17.059611
1,1,2,25,13,5,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.014112
2,2,2,5,4,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,22.001910
3,3,1,0,19,3,0,0,0,0,0,1,0,0,0,0,0,2,1,0,0,1,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26.003901
4,4,3,15,8,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17.028908


In [13]:
X_train_ok.loc[4][1:]

count_floors_pre_eq              3.000000
age                             30.000000
area_percentage                  8.000000
height_percentage                9.000000
has_superstructure_adobe_mud     1.000000
                                  ...    
legal_ownership_status_a         0.000000
legal_ownership_status_w         0.000000
legal_ownership_status_r         0.000000
legal_ownership_status_v         1.000000
geopos                          11.013101
Name: 4, Length: 66, dtype: float64

In [10]:
np.linalg.norm(X_train_ok.loc[4][1:])

34.428017723723045

In [20]:
X_pred_len=[]

for i in X_pred_ok.index:
    a=np.linalg.norm(X_pred_ok.loc[i][1:])
    X_pred_len.append(a)





In [21]:
X_train_len=[]

for i in X_pred_ok.index:
    a=np.linalg.norm(X_train_ok.loc[i][1:])
    X_train_len.append(a)

In [37]:
f=open(basedir+"/_EarthQuake/difference.csv"")
epsilon=0.001
k=0
for i,Xpi in enumerate(X_pred_len):
    for j,Xti  in enumerate(X_train_len):
        if abs(Xpi-Xti)<epsilon:
            delta=np.linalg.norm(X_pred_ok.loc[i][1:]-X_train_ok.loc[j][1:])
            if delta<6:
                k+=1
                if k>500:
                    print(k,i,j)
                print (f"{i},{j},{delta}", file=f)
f.close()
            

0 15506 2.4501645360189594
0 49275 1.4142135623731096
0 55389 2.0000004922691605
0 55926 2.000000495538502
2 61299 5.814059682014497
4 2090 2.0000000000055222
4 29494 3.8000000301963155e-07
4 50937 2.0000000000098628
4 65523 1.4142142571533354
7 1768 5.8530306602737285
8 2267 1.3969999983487469e-06
8 8471 0.0
8 43091 4.609999997740033e-06
8 49573 1.414213562373679
8 64922 2.449490035231106
8 76337 5.653619672073623
9 47608 3.741657472535673
9 50435 4.79143632024109
10 10105 3.000907601160223
12 6032 4.730334587694234
12 13532 2.449489742783178
12 30309 4.233761802434087
17 1809 2.8284271247491
19 5733 3.9510438915716333
19 36437 2.4514057816164385
19 48980 2.828527210436803
22 48003 5.395324151942346
22 84757 5.57759613542853
26 1694 4.690771887168832
26 38257 5.100373024105588
31 40716 3.3166786745734322
31 85387 3.162505992333924
32 932 1.414213562373111
32 7339 1.4142135623830376
32 8581 1.4142135623865764
32 10860 1.4142135623865764
32 31108 1.4142135623985672
32 35175 1.4142135623

KeyboardInterrupt: ignored

In [15]:


for i in X_pred_ok.index:
    min=5
    for j in range(260000):
        
        a=np.linalg.norm(X_train_ok.loc[j][1:]-X_pred_ok.loc[i][1:])
        if a < min:
            print(f"Adatok: pred:{i:8}, trean:{j:7},{a:8}")
            print(f"{i};{j},{a:16}", file=f)
            min=a
            if a==0:
                exit

f.close()




Adatok: pred:       0, trean:     10,3.1627101324897566
Adatok: pred:       0, trean:    224,2.2364186112152433
Adatok: pred:       0, trean:    254,1.4152634887509186
Adatok: pred:       0, trean:   2676,1.4148793035460652
Adatok: pred:       0, trean:   3063,1.4143769915896964
Adatok: pred:       0, trean:   3513,0.027893074999997935
Adatok: pred:       0, trean:  17537,0.02010024599999838
Adatok: pred:       0, trean:  39804,0.01969907999999876
Adatok: pred:       0, trean:  72865,0.006196126999999052


KeyboardInterrupt: ignored